In [2]:
import pandas as pd  # импортируем нужные библиотеки 
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet

In [3]:
df = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv', sep=',') #считываем данные из файла

In [3]:
df

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


In [4]:
df.info() # проанализируем датасет - видно, что присутствуют текстовые поля, преобразуем их

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [4]:
df = pd.concat([pd.get_dummies(df['Gender'], prefix='Gender', dtype='float64'), df], axis=1) # изменили пол
df.drop("Gender", axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender_Female                   2111 non-null   float64
 1   Gender_Male                     2111 non-null   float64
 2   Age                             2111 non-null   float64
 3   Height                          2111 non-null   float64
 4   Weight                          2111 non-null   float64
 5   family_history_with_overweight  2111 non-null   object 
 6   FAVC                            2111 non-null   object 
 7   FCVC                            2111 non-null   float64
 8   NCP                             2111 non-null   float64
 9   CAEC                            2111 non-null   object 
 10  SMOKE                           2111 non-null   object 
 11  CH2O                            2111 non-null   float64
 12  SCC                             21

In [6]:
df['family_history_with_overweight'].value_counts() # видно, что тут только 2 значения, используем get_dummies

yes    1726
no      385
Name: family_history_with_overweight, dtype: int64

In [5]:
df = pd.concat([pd.get_dummies(df['family_history_with_overweight'], prefix='fam_his_ow', dtype='float64'), df], axis=1)
df.drop("family_history_with_overweight", axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fam_his_ow_no   2111 non-null   float64
 1   fam_his_ow_yes  2111 non-null   float64
 2   Gender_Female   2111 non-null   float64
 3   Gender_Male     2111 non-null   float64
 4   Age             2111 non-null   float64
 5   Height          2111 non-null   float64
 6   Weight          2111 non-null   float64
 7   FAVC            2111 non-null   object 
 8   FCVC            2111 non-null   float64
 9   NCP             2111 non-null   float64
 10  CAEC            2111 non-null   object 
 11  SMOKE           2111 non-null   object 
 12  CH2O            2111 non-null   float64
 13  SCC             2111 non-null   object 
 14  FAF             2111 non-null   float64
 15  TUE             2111 non-null   float64
 16  CALC            2111 non-null   object 
 17  MTRANS          2111 non-null   o

In [6]:
df['FAVC'].value_counts()

df = pd.concat([pd.get_dummies(df['FAVC'], prefix='FAVC', dtype='float64'), df], axis=1)
df.drop("FAVC", axis=1, inplace=True)
df

,FAVC_no,FAVC_yes,fam_his_ow_no,fam_his_ow_yes,Gender_Female,Gender_Male,Age,Height,Weight,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,1.0,0.0,0.0,1.0,1.0,0.0,21.000000,1.620000,64.000000,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,1.0,0.0,0.0,1.0,1.0,0.0,21.000000,1.520000,56.000000,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,1.0,0.0,0.0,1.0,0.0,1.0,23.000000,1.800000,77.000000,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,1.0,0.0,1.0,0.0,0.0,1.0,27.000000,1.800000,87.000000,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,1.0,0.0,1.0,0.0,0.0,1.0,22.000000,1.780000,89.800000,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,0.0,1.0,0.0,1.0,1.0,0.0,20.976842,1.710730,131.408528,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,0.0,1.0,0.0,1.0,1.0,0.0,21.982942,1.748584,133.742943,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,0.0,1.0,0.0,1.0,1.0,0.0,22.524036,1.752206,133.689352,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,0.0,1.0,0.0,1.0,1.0,0.0,24.361936,1.739450,133.346641,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


In [9]:
df['CAEC'].value_counts() # видно что тут несколько значених, их не так много, поэтому можно использовать map


Sometimes     1765
Frequently     242
Always          53
no              51
Name: CAEC, dtype: int64

In [7]:
df["CAEC"] = df["CAEC"].map({"Sometimes": 1.0, "Frequently": 2, "Always": 3, "no": 0})
df['CAEC'].value_counts()


1.0    1765
2.0     242
3.0      53
0.0      51
Name: CAEC, dtype: int64

In [8]:
df['SMOKE'].value_counts()
df = pd.concat([pd.get_dummies(df['SMOKE'], prefix='SMOKE', dtype='float64'), df], axis=1)
df.drop("SMOKE", axis=1, inplace=True)


df['SCC'].value_counts()
df = pd.concat([pd.get_dummies(df['SCC'], prefix='SCC', dtype='float64'), df], axis=1)
df.drop("SCC", axis=1, inplace=True)


df['CALC'].value_counts()
df["CALC"] = df["CALC"].map({"Sometimes": 1.0, "Frequently": 2, "Always": 3, "no": 0})


df['MTRANS'].value_counts()
df["MTRANS"] = df["MTRANS"].map(
    {"Public_Transportation": 0.0, "Automobile": 1, "Walking": 2, "Motorbike": 3, "Bike": 4})


df['NObeyesdad'].value_counts()
df["NObeyesdad"] = df["NObeyesdad"].map(
    {"Obesity_Type_I": 4,  # тип ожирения
     "Obesity_Type_II": 5,
     "Obesity_Type_III": 6,
     "Overweight_Level_I": 2,  # Уровень избыточного веса
     "Overweight_Level_II": 3,
     "Normal_Weight": 1,
     "Insufficient_Weight": 0.0})  # недостаточный вес

df.info()
# мы перекодировали все текстовые поля, ненулевых значений нет, датасет готов

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SCC_no          2111 non-null   float64
 1   SCC_yes         2111 non-null   float64
 2   SMOKE_no        2111 non-null   float64
 3   SMOKE_yes       2111 non-null   float64
 4   FAVC_no         2111 non-null   float64
 5   FAVC_yes        2111 non-null   float64
 6   fam_his_ow_no   2111 non-null   float64
 7   fam_his_ow_yes  2111 non-null   float64
 8   Gender_Female   2111 non-null   float64
 9   Gender_Male     2111 non-null   float64
 10  Age             2111 non-null   float64
 11  Height          2111 non-null   float64
 12  Weight          2111 non-null   float64
 13  FCVC            2111 non-null   float64
 14  NCP             2111 non-null   float64
 15  CAEC            2111 non-null   float64
 16  CH2O            2111 non-null   float64
 17  FAF             2111 non-null   f

In [9]:
y = df["Weight"]  # вытаскивает(удаляет из таблицы) - мы должны спрогнозировать это значение
df.drop("Weight", axis=1, inplace=True)
x = df

lr = LinearRegression()
lr.fit(x, y)
print(lr.score(x, y))
print(np.mean(cross_val_score(lr, x, y, cv=5)))

0.9571395485789886
0.7985726126799377


In [10]:
# попробуем усложнить модель
# масштабирование переменных
std_Scaler = StandardScaler()
mm_Scaler = MinMaxScaler()

std_Scaler.fit(x)  # обучение
x_std = std_Scaler.transform(x)  # массив numpy
x_mm = mm_Scaler.fit_transform(x)

k = cross_val_score(lr, x_mm, y, cv=5)
kk = cross_val_score(lr, x_std, y, cv=5)
print(np.mean(k))
print(np.mean(kk))

0.7998486709381052
0.8000536040465617


In [11]:
# полином. регрессия
poly_r = Pipeline([('poly', PolynomialFeatures(degree=1)), ('scaler', StandardScaler()), ('linear', LinearRegression())])
param_grid = {'poly__degree': range(1, 4)}
grid_l = GridSearchCV(poly_r, param_grid=param_grid, cv=5)
grid_l.fit(x, y)
print(grid_l.best_score_)  # наилуч. рез.
print(grid_l.best_estimator_)  # параметры при кот. наилуч.

0.7987374844934385
Pipeline(steps=[('poly', PolynomialFeatures(degree=1)),
                ('scaler', StandardScaler()), ('linear', LinearRegression())])


In [12]:
l1 = Lasso(alpha=0.001)  # дообучить - уменьшить альфа - я уменьшала постепенно с 10 до 0.001. Если уменьшать дальше, то резульат будет примерно таким же
print(np.mean(cross_val_score(l1, x, y, cv=5)))
l2 = Ridge(alpha=0.001)
print(np.mean(cross_val_score(l2, x, y, cv=5)))
l3 = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=100000)
#(alpha=0.1, l1_ratio=0.8, max_iter=100000)   0.5860548493960487
print(np.mean(cross_val_score(l3, x, y, cv=5)))

0.7986256239821851
0.7985672180271728
0.5540813432407014


In [18]:
poly_en = Pipeline([('poly', PolynomialFeatures()), ('scaler', MinMaxScaler()), ('el_net', ElasticNet(max_iter=1000))])
param_grid2 = {'el_net__l1_ratio': [1, 0.1, 0.3, 0.5, 0.8], 'el_net__alpha': [0.03, 0.01, 0.05, 0.08 ],
               'poly__degree': [1], 'el_net__max_iter': [1000]}
#при степении 2 и больше нужно больщое кол-во итераций и это очень долго считается
grid_en_poly = GridSearchCV(poly_en, param_grid=param_grid2, cv=5)
grid_en_poly.fit(x, y)
print('grid_en.best_score_', grid_en_poly.best_score_)
print('grid_en.best_params_', grid_en_poly.best_params_)
#{'el_net__alpha': 0.01, 'el_net__l1_ratio': 1}  0.7987353230096146  {'l1_ratio': [0.1, 0.01, 0.001, 1], 'alpha': [0.1, 0.01, 0.001, 1]}
# тот же самый результат при   'el_net__l1_ratio': [1, 0.1, 0.3, 0.5, 0.8], 'el_net__alpha': [0.03, 0.01, 0.05, 0.08 ]

grid_en.best_score_ 0.7987353230096146
grid_en.best_params_ {'el_net__alpha': 0.01, 'el_net__l1_ratio': 1, 'el_net__max_iter': 1000, 'poly__degree': 1}


In [22]:
poly_rid = Pipeline([('poly', PolynomialFeatures()), ('scaler',MinMaxScaler()), ('el_net', Ridge(max_iter=1000))])
# minmaxscaler часто даёт какие то ошибки, поэтому я его не использую
param_grid3 = {'el_net__alpha': [0.0006,0.0007,0.0008, 0.0009]}
#{'el_net__alpha': np.linspace(0.0001, 0.9999, 10)}     0.8416241859534646 alpha=0.0001
# {'el_net__alpha': np.linspace(0.00001, 0.0001, 10)}   0.8416241859534646 alpha=0.0001
# {'el_net__alpha': [0.0001, 0.0003,0.0005,0.0008]}     0.8427512364755719  alpha=0.0008
# {'el_net__alpha': [0.0006,0.0007,0.0008, 0.0009]}     0.8428696352922138 {'el_net__alpha': 0.0009}
grid_rid = GridSearchCV(poly_rid, param_grid=param_grid3, cv=5)
grid_rid.fit(x, y)
print('grid_en.best_score_', grid_rid.best_score_)
print('grid_en.best_params_', grid_rid.best_params_)

grid_en.best_score_ 0.8428696352922138
grid_en.best_params_ {'el_net__alpha': 0.0009}


In [24]:
poly_las = Pipeline([('poly', PolynomialFeatures()), ('scaler', MinMaxScaler()), ('el_net', Lasso(max_iter=10000))])
param_grid4 = {'el_net__alpha': [0.01, 0.1]}
# {'el_net__alpha': [1, 0.1]}, 0.1  0.7673290626728526 maxiter=5000
# {'el_net__alpha': [0.01, 0.1]}  0.01  0.8230501060917639 maxiter=10000
grid_las = GridSearchCV(poly_las, param_grid=param_grid4, cv=5)
grid_las.fit(x, y)
print('grid_en.best_score_', grid_las.best_score_)
print('grid_en.best_params_', grid_las.best_params_)

grid_en.best_score_ 0.8230501060917639
grid_en.best_params_ {'el_net__alpha': 0.01}


In [25]:
# хорошие резульаты модель показала при использовании полинома с Ridge (best_score_ = 0.8428696352922138)